## Distributed hyperparameter tuning with Hyperopt + SparkTrials

- Dataset: [Credit Card Fraud Detection, Worldline & Machine Learning Group of ULB](http://mlg.ulb.ac.be/)
  - Anonymized credit card transactions labeled as fraudulent or genuine
  - Hosted on Databricks at `/databricks-datasets/credit-card-fraud`

- Model: `sklearn.ensemble.RandomForestClassifier`
- Tuning: Hyperopt with `SparkTrials`
  - An open-source tuning library that employs Tree of Parzien Estimators (TPE)
  - Spark backend for distributed tuning

In [0]:
import mlflow
import pandas as pd

from hyperopt import hp, tpe, fmin, STATUS_OK, Trials, SparkTrials
from hyperopt.pyll.base import scope

from pyspark.ml.functions import vector_to_array

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [0]:
mlflow.sklearn.autolog(disable=True)

#### Load and prepare the data

In [0]:
spark_df = spark.read.parquet("/databricks-datasets/credit-card-fraud/data")
spark_df = spark_df.withColumn("pca", vector_to_array("pcaVector"))

pca_length = 28
pca_columns = [spark_df.pca[i] for i in range(pca_length)]

data = spark_df.select(["time", "amountRange", "label"] + pca_columns).toPandas()
data = data.fillna(data.mean())
target_col = "label"

In [0]:
data.head()

Out[4]:

,time,amountRange,label,pca[0],pca[1],pca[2],pca[3],pca[4],pca[5],pca[6],pca[7],pca[8],pca[9],pca[10],pca[11],pca[12],pca[13],pca[14],pca[15],pca[16],pca[17],pca[18],pca[19],pca[20],pca[21],pca[22],pca[23],pca[24],pca[25],pca[26],pca[27]
0,52972.0,2,0,-0.775461,0.709595,1.610122,1.231792,0.316178,0.114410,0.410964,0.048246,0.016042,0.590293,1.069637,-0.097002,-2.053954,0.296434,-0.242126,-1.103983,0.438081,-0.090098,0.972525,-0.009525,-0.031317,0.289930,-0.215881,0.196379,-0.267693,-0.298693,0.011195,-0.023320
1,41768.0,6,0,0.873554,-1.377510,1.072031,0.898335,-1.377677,1.134118,-1.049505,0.303593,0.391175,0.306457,-1.743321,0.496823,0.582594,-1.042981,-1.093048,-2.158048,0.552883,0.941660,-0.878166,-0.188112,-0.206446,-0.020860,-0.294526,-0.419986,0.530462,-0.095314,0.102470,0.061393
2,40769.0,7,0,0.890897,-0.528187,-0.678654,0.168701,-0.141127,-1.037345,0.786720,-0.463099,-0.231256,-0.210527,-0.579607,0.320749,0.739513,0.385612,0.760072,0.056286,-0.365932,-0.553207,0.225507,0.469012,0.072491,-0.283583,-0.367602,-0.003742,0.548136,1.073835,-0.156020,0.029210
3,40682.0,3,0,-0.572954,0.458246,0.239598,-1.561953,2.688063,3.529501,0.386101,0.773474,0.061670,-0.425875,-0.594523,-0.149708,-0.368630,-0.285347,-0.600322,0.106861,-0.793363,-0.208004,0.082330,0.229461,-0.290700,-0.627817,-0.206940,1.021907,0.191261,0.219703,0.122007,-0.062997
4,50032.0,7,0,-2.053059,0.504530,-0.111156,-1.040738,-2.684590,0.048439,2.152220,0.497990,-0.802817,-1.262772,1.101034,0.617550,-0.478062,0.968829,-0.337551,1.020513,-0.476524,0.048632,-0.389469,-0.142086,-0.051183,-0.370120,0.583370,0.556771,-0.250298,0.631668,0.103110,-0.186396


In [0]:
# Split into train and validation sets
X = data.drop([target_col], axis=1)
y = data[target_col]

X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y)

### Single-node hyperparameter tuning

#### Define search space and objective function

In [0]:
# Define the hyperparameter search space
space = {
  "n_estimators": scope.int(hp.quniform("n_estimators", 50, 150, 1)),
  "criterion": hp.choice("criterion", ["gini", "entropy"]),
  "min_samples_leaf": scope.int(hp.quniform("min_samples_leaf", 1, 20, 1)),
  "min_samples_split": hp.uniform("min_samples_split", 0, 1),
}

n_evals = 200

In [0]:
def objective(hyperparameters):
  # Instantiate the model with hyperparameters
  model = RandomForestClassifier(**hyperparameters)

  # Train the model
  model.fit(X_train, y_train)

  # Evaluate the learned model
  val_pred = model.predict(X_val)
  val_f1_score = f1_score(y_val, val_pred)

  # Use negative F1 score as our loss metric
  return {"loss": -val_f1_score, "status": STATUS_OK}

#### Train a `RandomForestClassifier` with Hyperopt

In [0]:
trials = Trials()
best = fmin(
  fn=objective,
  space=space,
  algo=tpe.suggest,
  max_evals=n_evals,
  trials=trials
)

best_f1 = sorted(trials.results, key=lambda result: result["loss"])[0]["loss"] * -1

0%| | 0/200 [00:00<?, ?trial/s, best loss=?] 0%| | 1/200 [00:03<10:52, 3.28s/trial, best loss: -0.0] 1%| | 2/200 [00:05<09:42, 2.94s/trial, best loss: -0.0] 2%|▏ | 3/200 [00:08<09:22, 2.86s/trial, best loss: -0.0] 2%|▏ | 4/200 [01:58<1:55:04, 35.23s/trial, best loss: -0.0] 2%|▎ | 5/200 [05:49<5:05:26, 93.98s/trial, best loss: -0.6915887850467289] 3%|▎ | 6/200 [07:39<5:18:35, 98.54s/trial, best loss: -0.6915887850467289] 4%|▎ | 7/200 [08:49<4:50:17, 90.24s/trial, best loss: -0.6915887850467289] 4%|▍ | 8/200 [10:32<5:00:13, 93.82s/trial, best loss: -0.7004608294930876] 4%|▍ | 9/200 [10:34<3:30:50, 66.23s/trial, best loss: -0.7004608294930876] 5%|▌ | 10/200 [14:09<5:51:40, 111.06s/trial, best loss: -0.7037037037037038] 6%|▌ | 11/200 [17:18<7:02:52, 134.24s/trial, best loss: -0.7037037037037038] 6%|▌ | 12/200 [19:21<6:50:43, 131.08s/trial, best loss: -0.7037037037037038] 6%|▋ | 13/200 [23:01<8:11:46, 157.79s/trial, best loss: -0.7037037037037038] 7%|▋ | 14/200 [24:48<7:21:40, 142.48s/trial, best loss: -0.8189655172413793] 8%|▊ | 15/200 [27:38<7:44:58, 150.80s/trial, best loss: -0.8189655172413793] 8%|▊ | 16/200 [27:40<5:24:58, 105.97s/trial, best loss: -0.8189655172413793] 8%|▊ | 17/200 [29:07<5:06:35, 100.52s/trial, best loss: -0.8189655172413793] 9%|▉ | 18/200 [29:10<3:35:29, 71.04s/trial, best loss: -0.8189655172413793]  10%|▉ | 19/200 [30:35<3:47:06, 75.29s/trial, best loss: -0.8189655172413793] 10%|█ | 20/200 [33:07<4:54:46, 98.26s/trial, best loss: -0.8189655172413793] 10%|█ | 21/200 [34:34<4:42:50, 94.81s/trial, best loss: -0.8189655172413793] 11%|█ | 22/200 [37:18<5:43:01, 115.63s/trial, best loss: -0.8189655172413793] 12%|█▏ | 23/200 [38:46<5:17:03, 107.48s/trial, best loss: -0.8189655172413793] 12%|█▏ | 24/200 [41:07<5:44:15, 117.36s/trial, best loss: -0.8189655172413793] 12%|█▎ | 25/200 [43:26<6:01:34, 123.97s/trial, best loss: -0.8189655172413793] 13%|█▎ | 26/200 [44:51<5:25:31, 112.25s/trial, best loss: -0.8189655172413793] 14%|█▎ | 27/200 [47:31<6:04:55, 126.56s/trial, best loss: -0.8189655172413793] 14%|█▍ | 28/200 [49:15<5:43:41, 119.89s/trial, best loss: -0.8189655172413793] 14%|█▍ | 29/200 [50:26<4:59:59, 105.26s/trial, best loss: -0.8189655172413793] 15%|█▌ | 30/200 [52:11<4:57:26, 104.98s/trial, best loss: -0.8189655172413793] 16%|█▌ | 31/200 [55:46<6:29:03, 138.13s/trial, best loss: -0.8189655172413793] 16%|█▌ | 32/200 [55:48<4:31:55, 97.12s/trial, best loss: -0.8189655172413793]  16%|█▋ | 33/200 [55:50<3:10:52, 68.58s/trial, best loss: -0.8189655172413793] 17%|█▋ | 34/200 [57:55<3:56:46, 85.58s/trial, best loss: -0.8189655172413793] 18%|█▊ | 35/200 [1:00:06<4:33:05, 99.30s/trial, best loss: -0.8189655172413793] 18%|█▊ | 36/200 [1:00:08<3:11:10, 69.94s/trial, best loss: -0.8189655172413793] 18%|█▊ | 37/200 [1:01:26<3:16:41, 72.40s/trial, best loss: -0.8189655172413793] 19%|█▉ | 38/200 [1:03:04<3:36:46, 80.29s/trial, best loss: -0.8189655172413793] 20%|█▉ | 39/200 [1:05:33<4:30:40, 100.87s/trial, best loss: -0.8189655172413793] 20%|██ | 40/200 [1:07:14<4:28:42, 100.77s/trial, best loss: -0.8189655172413793] 20%|██ | 41/200 [1:09:04<4:34:30, 103.59s/trial, best loss: -0.8189655172413793] 21%|██ | 42/200 [1:09:05<3:11:54, 72.88s/trial, best loss: -0.8189655172413793]  22%|██▏ | 43/200 [1:10:24<3:15:17, 74.63s/trial, best loss: -0.8189655172413793] 22%|██▏ | 44/200 [1:12:11<3:39:33, 84.45s/trial, best loss: -0.8189655172413793] 22%|██▎ | 45/200 [1:13:22<3:27:26, 80.30s/trial, best loss: -0.8189655172413793] 23%|██▎ | 46/200 [1:13:23<2:25:24, 56.65s/trial, best loss: -0.8189655172413793] 24%|██▎ | 47/200 [1:14:30<2:31:46, 59.52s/trial, best loss: -0.8189655172413793] 24%|██▍ | 48/200 [1:17:43<4:12:49, 99.80s/trial, best loss: -0.8189655172413793] 24%|██▍ | 49/200 [1:19:54<4:34:25, 109.04s/trial, best loss: -0.8189655172413793] 25%|██▌ | 50/200 [1:19:56<3:12:01, 76.81s/trial, best loss: -0.8189655172413793]  26%|██▌ | 51/200 [1:21:19<3:16:00, 78.93s/trial, best loss

In [0]:
best["F1 Score"] = best_f1
best["criterion"] = "gini" if best["criterion"] == 0 else "entropy"
display(pd.DataFrame(best, index=[0]))

criterion,min_samples_leaf,min_samples_split,n_estimators,F1 Score
gini,7.0,1.1731443489267082E-4,83.0,0.8225108225108225


### Distributed hyperparameter tuning

#### Broadcast data

In [0]:
# Broadcast the data
X_train_bc = sc.broadcast(X_train)
y_train_bc = sc.broadcast(y_train)
X_val_bc = sc.broadcast(X_val)
y_val_bc = sc.broadcast(y_val)

#### Train a `RandomForestClassifier` with Hyperopt + `SparkTrials`

In [0]:
def objective(hyperparameters):
  # Get the broadcasted variables
  X_train = X_train_bc.value
  y_train = y_train_bc.value
  X_val = X_val_bc.value
  y_val = y_val_bc.value
  
  # Instantiate the model with hyperparameters
  model = RandomForestClassifier(**hyperparameters)

  # Train the model
  model.fit(X_train, y_train)

  # Evaluate the learned model
  val_pred = model.predict(X_val)
  val_f1_score = f1_score(y_val, val_pred)

  # Use negative F1 score as our loss metric
  return {"loss": -val_f1_score, "status": STATUS_OK}

In [0]:
trials = SparkTrials(parallelism=32)
best = fmin(
  fn=objective,
  space=space,
  algo=tpe.suggest,
  max_evals=n_evals,
  trials=trials
)

best_f1 = sorted(trials.results, key=lambda result: result["loss"])[0]["loss"] * -1

Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.
 0%| | 0/200 [00:00<?, ?trial/s, best loss=?] 1%| | 2/200 [00:08<13:33, 4.11s/trial, best loss: -0.0] 2%|▏ | 3/200 [00:11<12:49, 3.91s/trial, best loss: -0.0] 2%|▏ | 4/200 [00:17<14:29, 4.44s/trial, best loss: -0.0] 2%|▎ | 5/200 [00:26<19:00, 5.85s/trial, best loss: -0.0] 3%|▎ | 6/200 [00:29<16:35, 5.13s/trial, best loss: -0.0] 4%|▎ | 7/200 [00:31<12:41, 3.94s/trial, best loss: -0.0] 4%|▍ | 9/200 [00:32<09:20, 2.94s/trial, best loss: -0.0] 6%|▌ | 11/200 [00:33<07:01, 2.23s/trial, best loss: -0.0] 6%|▌ | 12/200 [00:34<05:58, 1.91s/trial, best loss: -0.0] 8%|▊ | 15/200 [00:40<05:53, 1.91s/trial, best loss: -0.0] 8%|▊ | 16/200 [00:41<05:09, 1.68s/trial, best loss: -0.0] 8%|▊ | 17/200 [00:43<05:41, 1.86s/trial, best loss: -0.0] 10%|▉ | 19/200 [00:44<04:27, 1.48s/trial, best loss: -0.0] 10%|█ | 21/200 [00:49<05:08, 1.72s/trial, best loss: -0.0] 11%|█ | 22/200 [00:50<04:36, 1.55s/trial, best loss: -0.0] 12%|█▏ | 23/200 [00:51<04:13, 1.43s/trial, best loss: -0.0] 12%|█▏ | 24/200 [00:54<04:57, 1.69s/trial, best loss: -0.0] 12%|█▎ | 25/200 [00:55<04:28, 1.53s/trial, best loss: -0.0] 14%|█▎ | 27/200 [00:56<03:35, 1.25s/trial, best loss: -0.0] 14%|█▍ | 29/200 [00:59<03:58, 1.39s/trial, best loss: -0.0] 15%|█▌ | 30/200 [01:03<05:42, 2.01s/trial, best loss: -0.0] 16%|█▌ | 31/200 [01:04<04:58, 1.76s/trial, best loss: -0.0] 16%|█▌ | 32/200 [01:05<04:25, 1.58s/trial, best loss: -0.0] 16%|█▋ | 33/200 [01:07<04:59, 1.79s/trial, best loss: -0.0] 17%|█▋ | 34/200 [01:11<06:20, 2.29s/trial, best loss: -0.0] 18%|█▊ | 35/200 [01:16<08:12, 2.98s/trial, best loss: -0.0] 18%|█▊ | 36/200 [01:17<06:37, 2.43s/trial, best loss: -0.0] 18%|█▊ | 37/200 [01:20<07:24, 2.73s/trial, best loss: -0.0] 19%|█▉ | 38/200 [01:21<06:04, 2.25s/trial, best loss: -0.0] 20%|█▉ | 39/200 [01:22<05:09, 1.93s/trial, best loss: -0.0] 20%|██ | 40/200 [01:24<04:30, 1.69s/trial, best loss: -0.0] 20%|██ | 41/200 [01:26<04:56, 1.87s/trial, best loss: -0.0] 21%|██ | 42/200 [01:27<04:21, 1.66s/trial, best loss: -0.0] 22%|██▏ | 43/200 [01:28<03:55, 1.50s/trial, best loss: -0.0] 22%|██▏ | 44/200 [01:30<04:29, 1.73s/trial, best loss: -0.0] 22%|██▎ | 45/200 [01:32<04:00, 1.55s/trial, best loss: -0.0] 23%|██▎ | 46/200 [01:34<04:33, 1.78s/trial, best loss: -0.0] 24%|██▎ | 47/200 [01:36<04:54, 1.93s/trial, best loss: -0.0] 24%|██▍ | 48/200 [01:39<05:58, 2.36s/trial, best loss: -0.0] 24%|██▍ | 49/200 [01:43<06:43, 2.67s/trial, best loss: -0.0] 25%|██▌ | 50/200 [01:47<08:04, 3.23s/trial, best loss: -0.0] 26%|██▌ | 51/200 [01:51<08:08, 3.28s/trial, best loss: -0.0] 26%|██▌ | 52/200 [01:54<08:11, 3.32s/trial, best loss: -0.0] 26%|██▋ | 53/200 [02:03<12:21, 5.05s/trial, best loss: -0.0] 27%|██▋ | 54/200 [02:09<12:44, 5.24s/trial, best loss: -0.0] 28%|██▊ | 55/200 [02:10<09:42, 4.02s/trial, best loss: -0.0] 28%|██▊ | 57/200 [02:14<07:55, 3.33s/trial, best loss: -0.3178807947019867] 29%|██▉ | 58/200 [02:15<06:13, 2.63s/trial, best loss: -0.3178807947019867] 30%|██▉ | 59/200 [02:20<07:53, 3.36s/trial, best loss: -0.3178807947019867] 30%|███ | 60/200 [02:22<06:54, 2.96s/trial, best loss: -0.3178807947019867] 30%|███ | 61/200 [02:24<06:18, 2.73s/trial, best loss: -0.6857142857142856] 31%|███ | 62/200 [02:26<05:47, 2.52s/trial, best loss: -0.6857142857142856] 32%|███▏ | 63/200 [02:31<07:27, 3.27s/trial, best loss: -0.6857142857142856] 32%|███▏ | 64/200 [02:33<06:33, 2.89s/trial, best loss: -0.6857142857142856] 32%|███▎ | 65/200 [02:34<05:14, 2.33s/trial, best l

In [0]:
best["F1 Score"] = best_f1
best["criterion"] = "gini" if best["criterion"] == 0 else "entropy"
display(pd.DataFrame(best, index=[0]))

criterion,min_samples_leaf,min_samples_split,n_estimators,F1 Score
gini,10.0,3.2316495187868083E-4,60.0,0.7948717948717948
